# Data Preparation

In [1]:
from darts import TimeSeries
import pandas as pd
import numpy as np


In [81]:
def prepare_BTC_data(file_path):
    df = pd.read_csv(file_path)
    df  =df.reset_index()
    df = df.rename(columns={"Date": "ds",
                            "Close": "y",
                            "High": "High",
                            "Low": "Low",
                            "Open": "Open"})
    df = df.drop(columns=['Volume', 'Dividends', "Stock Splits"], axis=1)
    df['ds'] = pd.to_datetime(df['ds'])

    return df

def split_data(df):
    latest_date = pd.Timestamp(df['ds'].max())
    last_year_data = latest_date.year -1

    test= df[(df['ds'].dt.year == last_year_data)]
    test_start = pd.Timestamp(test['ds'].min())

    val_start = test_start - pd.DateOffset(months=9)
    val_end = test_start - pd.DateOffset(days=1)
    val = df[(df['ds'] >= val_start) & (df['ds'] <= val_end)]
    optimization = val.iloc[-14:]
    val = val.iloc[:-14]

    train_end = val_start - pd.DateOffset(days=1)
    train = df[(df['ds'] <= train_end)]
    return test,val,train,optimization

def dataframe_to_timeseries(
    df, 
    time_col='ds', 
    value_cols=['y'], 
    freq='D'
):
    return TimeSeries.from_dataframe(
        df,
        time_col=time_col,
        value_cols=value_cols,
        freq=freq
    )


def timeseries_to_dataframe(series, time_col='ds', value_col='y'):
    df = series.to_dataframe()
    df = df.reset_index()    
    return df

In [83]:
df = prepare_BTC_data("BTC-USD_2015-2025_daily.csv")
df.drop('index', axis= True, inplace = True)

In [85]:
test,val,train,optimization = split_data(df)

In [87]:
# Example usage:
train_series = dataframe_to_timeseries(train)
val_series = dataframe_to_timeseries(val)
test_series = dataframe_to_timeseries(test, value_cols='y')  # For single column as string
optimization_series = dataframe_to_timeseries(optimization, value_cols='y')  # For single column as string

The provided DatetimeIndex was associated with a timezone, which is currently not supported by xarray. To avoid unexpected behaviour, the tz information was removed. Consider calling `ts.time_index.tz_localize(UTC)` when exporting the results.To plot the series with the right time steps, consider setting the matplotlib.pyplot `rcParams['timezone']` parameter to automatically convert the time axis back to the original timezone.
The provided DatetimeIndex was associated with a timezone, which is currently not supported by xarray. To avoid unexpected behaviour, the tz information was removed. Consider calling `ts.time_index.tz_localize(UTC)` when exporting the results.To plot the series with the right time steps, consider setting the matplotlib.pyplot `rcParams['timezone']` parameter to automatically convert the time axis back to the original timezone.
The provided DatetimeIndex was associated with a timezone, which is currently not supported by xarray. To avoid unexpected behaviour, the 

In [89]:
train_series

<TimeSeries (DataArray) (ds: 2953, component: 1, sample: 1)>
array([[[  260.20199585]],

       [[  275.67001343]],

       [[  281.70199585]],

       ...,

       [[28348.44140625]],

       [[28033.5625    ]],

       [[28478.484375  ]]])
Coordinates:
  * ds         (ds) datetime64[ns] 2015-03-01 2015-03-02 ... 2023-03-31
  * component  (component) <U1 'y'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None
    metadata:           None

In [91]:
val_series

<TimeSeries (DataArray) (ds: 261, component: 1, sample: 1)>
array([[[28411.03515625]],

       [[28199.30859375]],

       [[27790.22070312]],

       [[28168.08984375]],

       [[28177.984375  ]],

       [[28044.140625  ]],

       [[27925.859375  ]],

       [[27947.79492188]],

       [[28333.05078125]],

       [[29652.98046875]],

...

       [[44166.6015625 ]],

       [[43725.984375  ]],

       [[43779.69921875]],

       [[41243.83203125]],

       [[41450.22265625]],

       [[42890.7421875 ]],

       [[43023.97265625]],

       [[41929.7578125 ]],

       [[42240.1171875 ]],

       [[41364.6640625 ]]])
Coordinates:
  * ds         (ds) datetime64[ns] 2023-04-01 2023-04-02 ... 2023-12-17
  * component  (component) <U1 'y'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None
    metadata:           None

In [95]:
test_series

<TimeSeries (DataArray) (ds: 366, component: 1, sample: 1)>
array([[[ 44167.33203125]],

       [[ 44957.96875   ]],

       [[ 42848.17578125]],

       [[ 44179.921875  ]],

       [[ 44162.69140625]],

       [[ 43989.1953125 ]],

       [[ 43943.09765625]],

       [[ 46970.50390625]],

       [[ 46139.73046875]],

       [[ 46627.77734375]],

...

       [[ 95104.9375    ]],

       [[ 94686.2421875 ]],

       [[ 98676.09375   ]],

       [[ 99299.1953125 ]],

       [[ 95795.515625  ]],

       [[ 94164.859375  ]],

       [[ 95163.9296875 ]],

       [[ 93530.2265625 ]],

       [[ 92643.2109375 ]],

       [[ 93429.203125  ]]])
Coordinates:
  * ds         (ds) datetime64[ns] 2024-01-01 2024-01-02 ... 2024-12-31
  * component  (component) <U1 'y'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None
    metadata:           None

In [93]:
optimization_series

<TimeSeries (DataArray) (ds: 14, component: 1, sample: 1)>
array([[[42623.5390625 ]],

       [[42270.52734375]],

       [[43652.25      ]],

       [[43869.15234375]],

       [[43997.90234375]],

       [[43739.54296875]],

       [[43016.1171875 ]],

       [[43613.140625  ]],

       [[42520.40234375]],

       [[43442.85546875]],

       [[42627.85546875]],

       [[42099.40234375]],

       [[42156.90234375]],

       [[42265.1875    ]]])
Coordinates:
  * ds         (ds) datetime64[ns] 2023-12-18 2023-12-19 ... 2023-12-31
  * component  (component) <U1 'y'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None
    metadata:           None

# Tuning the Model

In [123]:
import optuna
from darts.metrics import mape

def tuning_model(train_series,val_series,optimization_series):
    # 3. Define Optuna objective
    def objective(trial):
        params = {
            'input_chunk_length': trial.suggest_categorical
                                ('input_chunk_length', [30, 60, 90, 120, 150, 180]),            
            'output_chunk_length': 14,
            'num_stacks': trial.suggest_int('num_stacks', 1, 5),
            'num_blocks': trial.suggest_int('num_blocks', 1, 3),
            'num_layers': trial.suggest_int('num_layers', 1, 4),
            'layer_widths': trial.suggest_categorical('layer_widths', [32, 64, 128, 256]),
            'n_epochs': 20,
            'batch_size': trial.suggest_categorical('batch_size', [16, 32, 64]),
            'optimizer_kwargs': {'lr': trial.suggest_loguniform('lr', 1e-5, 1e-3)},
            'random_state': 42,
            'force_reset': True
        }
        model = NBEATSModel(**params)
        model.fit(train_series, val_series=val_series, verbose=False)
        pred = model.predict(n=14, series=val_series)
        return mape(optimization_series, pred)

    # 4. Run Optuna study in parallel
    study = optuna.create_study(
        direction='minimize',
        sampler=optuna.samplers.TPESampler(),
        pruner=optuna.pruners.MedianPruner(),
        study_name='nbeats_hyperparam'
    )
    study.optimize(objective, n_trials=20)

    # 5. Train final model on full data (train+val+test?) or up to test start
    best_params = study.best_params.copy()
    # Extract learning rate from best_params and wrap into optimizer_kwargs
    lr = best_params.pop('lr')
    optimizer_kwargs = {'lr': lr}

    return best_params,lr,optimizer_kwargs

In [125]:
best_params,lr,optimizer_kwargs = tuning_model(train_series,val_series,optimization_series)

[I 2025-04-30 20:38:26,606] A new study created in memory with name: nbeats_hyperparam
C:\Users\hp\AppData\Local\Temp\ipykernel_12452\3647473208.py:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'optimizer_kwargs': {'lr': trial.suggest_loguniform('lr', 1e-5, 1e-3)},
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer.fit` stopped: `max_epochs=20` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |                                                                                    | 0/? [00:00<…

[I 2025-04-30 20:39:41,005] Trial 0 finished with value: 4.62288313660423 and parameters: {'input_chunk_length': 60, 'num_stacks': 5, 'num_blocks': 1, 'num_layers': 2, 'layer_widths': 128, 'batch_size': 32, 'lr': 1.7781170197168758e-05}. Best is trial 0 with value: 4.62288313660423.
C:\Users\hp\AppData\Local\Temp\ipykernel_12452\3647473208.py:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'optimizer_kwargs': {'lr': trial.suggest_loguniform('lr', 1e-5, 1e-3)},
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer.fit` stopped: `max_epochs=20` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |                                                                                    | 0/? [00:00<…

[I 2025-04-30 20:40:27,642] Trial 1 finished with value: 1.6377446716666049 and parameters: {'input_chunk_length': 30, 'num_stacks': 1, 'num_blocks': 1, 'num_layers': 3, 'layer_widths': 64, 'batch_size': 16, 'lr': 9.062593634164278e-05}. Best is trial 1 with value: 1.6377446716666049.
C:\Users\hp\AppData\Local\Temp\ipykernel_12452\3647473208.py:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'optimizer_kwargs': {'lr': trial.suggest_loguniform('lr', 1e-5, 1e-3)},
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer.fit` stopped: `max_epochs=20` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


...

[I 2025-04-30 21:07:11,402] Trial 17 finished with value: 4.82197267299596 and parameters: {'input_chunk_length': 150, 'num_stacks': 2, 'num_blocks': 1, 'num_layers': 3, 'layer_widths': 128, 'batch_size': 16, 'lr': 7.455051965997211e-05}. Best is trial 1 with value: 1.6377446716666049.
C:\Users\hp\AppData\Local\Temp\ipykernel_12452\3647473208.py:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'optimizer_kwargs': {'lr': trial.suggest_loguniform('lr', 1e-5, 1e-3)},
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer.fit` stopped: `max_epochs=20` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |                                                                                    | 0/? [00:00<…

[I 2025-04-30 21:07:39,515] Trial 18 finished with value: 1.6299839724398215 and parameters: {'input_chunk_length': 30, 'num_stacks': 3, 'num_blocks': 1, 'num_layers': 3, 'layer_widths': 64, 'batch_size': 64, 'lr': 0.0005244296294036305}. Best is trial 18 with value: 1.6299839724398215.
C:\Users\hp\AppData\Local\Temp\ipykernel_12452\3647473208.py:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'optimizer_kwargs': {'lr': trial.suggest_loguniform('lr', 1e-5, 1e-3)},
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer.fit` stopped: `max_epochs=20` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |                                                                                    | 0/? [00:00<…

[I 2025-04-30 21:08:34,923] Trial 19 finished with value: 31.891388624799298 and parameters: {'input_chunk_length': 30, 'num_stacks': 3, 'num_blocks': 2, 'num_layers': 4, 'layer_widths': 64, 'batch_size': 64, 'lr': 1.0196478426311006e-05}. Best is trial 18 with value: 1.6299839724398215.


# Train the Model

In [131]:
best_params

{'input_chunk_length': 30,
 'num_stacks': 3,
 'num_blocks': 1,
 'num_layers': 3,
 'layer_widths': 64,
 'batch_size': 64}

In [161]:
from darts.models import NBEATSModel

def train_model(best_params,optimizer_kwargs,train_series,val_series):
    # Add fixed params
    final_params = {
        **best_params,
         'output_chunk_length':14,
        'n_epochs': 50,
        'force_reset': True,
        'random_state': 42,
        'optimizer_kwargs': optimizer_kwargs
    }
    # Optionally: full series for final training
    full_val = val_series.append(optimization_series)
    
    # Instantiate final model
    final_model = NBEATSModel(**final_params)
    # Fit the model
    model.fit(
        series=train_series,
        val_series=full_val,
        verbose=True
    )
    # Optionally: full series for final training
    full_ts = val_series.append(optimization_series)

    # 5. Predict
    forecast = model.predict(
        n=14,
        series=full_val,
        verbose=True
    )
    return forecast

In [163]:
forecast = train_model(best_params,optimizer_kwargs,train_series,val_series)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Attempting to retrain/fine-tune the model without resuming from a checkpoint. This is currently discouraged. Consider model `NBEATSModel.load_weights()` to load the weights for fine-tuning.


Sanity Checking: |                                                                               | 0/? [00:00<…

Training: |                                                                                      | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

...

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

Validation: |                                                                                    | 0/? [00:00<…

`Trainer.fit` stopped: `max_epochs=50` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |                                                                                    | 0/? [00:00<…

In [164]:
last_actual = optimization['y'].iloc[-1]
last_actual

42265.1875

In [165]:
def compute_directions(test_results, last_actual):
    """
    Computes the percentage of times the predicted and actual direction of movement match
    relative to a baseline value.
    """
    test_results = test_results.dropna(subset=['y', 'yhat'])
    predicted_dir = (test_results['yhat'] > last_actual).astype(int)
    actual_dir = (test_results['y'] > last_actual).astype(int)

    test_results ['predicted_dir'] =predicted_dir 
    test_results ['actual_dir'] =actual_dir 

    return test_results
    
def create_test_results(forecast,test,last_actual):
    forecast_df = timeseries_to_dataframe(forecast)
    forecast_df = forecast_df.rename(columns={'y': 'yhat'})

    test_results = test.copy()
    test_results['ds'] = test_results['ds'].dt.tz_convert('UTC')  # Add UTC timezone
    forecast_df['ds'] = forecast_df['ds'].dt.tz_localize('UTC')   # Add UTC timezone
    test_results = pd.merge(test_results, forecast_df, on="ds", how="inner")

    test_results = compute_directions(test_results, last_actual)

    return test_results

In [166]:
test_results = create_test_results(forecast,test,last_actual)

In [167]:
test_results

,ds,Open,High,Low,y,yhat,predicted_dir,actual_dir
0,2024-01-01 00:00:00+00:00,42280.234375,44175.437500,42214.976562,44167.332031,41865.039365,0,1
1,2024-01-02 00:00:00+00:00,44187.140625,45899.707031,44176.949219,44957.968750,42088.777893,0,1
2,2024-01-03 00:00:00+00:00,44961.601562,45503.242188,40813.535156,42848.175781,41570.112921,0,1
3,2024-01-04 00:00:00+00:00,42855.816406,44770.023438,42675.175781,44179.921875,41478.189702,0,1
4,2024-01-05 00:00:00+00:00,44192.980469,44353.285156,42784.718750,44162.691406,41906.633413,0,1
5,2024-01-06 00:00:00+00:00,44178.953125,44227.632812,43475.156250,43989.195312,41696.372772,0,1
6,2024-01-07 00:00:00+00:00,43998.464844,44495.570312,43662.230469,43943.097656,41549.793880,0,1
7,2024-01-08 00:00:00+00:00,43948.707031,47218.000000,43244.082031,46970.503906,41592.450147,0,1
8,2024-01-09 00:00:00+00:00,46987.640625,47893.699219,45244.714844,46139.730469,41431.222647,0,1
9,2024-01-10 00:00:00+00:00,46121.539062,47647.222656,44483.152344,46627.777344,41309.401102,0,1
